In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

### MATRIZ BCH

Esta matríz se ha obtenido del artículo científico: `Brightness Calculation in Digital Image Processing` y nos ayudará a calcular el brillo de una imagen.

In [ ]:
MATRIZ_toDEF = np.array([[0.2053, 0.7125, 0.4670],
                        [1.8537, -1.2797, -0.4429],
                        [-0.3655, 1.0120, -0.6014]])

print(MATRIZ_toDEF) 

In [ ]:
MATRIZ_toXYZ = np.array([[0.6712, 0.4955, 0.1540],
                        [0.7061, 0.0248, 0.5223],
                        [0.7689, -0.2556, -0.8645]])

### 🧮 Cálculo del Brillo utilizando la Métrica de Cohen  

La función `obtener_brillo` calcula el brillo de una imagen a partir de sus componentes en el espacio de color XYZ, utilizando la métrica de Cohen basada en el modelo BCH (Brightness, Chroma, Hue).  

**¿Cómo funciona?**  
- Transformación BCH:  
   - Se toma la imagen en sus componentes **X, Y, Z** (R, G, B) y se aplica la matriz auxiliar para obtener nuevas representaciones **D, E, F**.    

- Cálculo brillo:
   - A partir de los valores **D, E, F**, se calcula el brillo **B** como la norma euclidiana en este espacio:  
     
     ```math
     B = \sqrt{D^2 + E^2 + F^2}
     ```
   - Luego, se obtiene el **brillo promedio**  de toda la imagen.  


In [ ]:
def calc_metricas_cohen(X, Y, Z):
    alto, ancho = X.shape
    # Stack y reshape en una matriz (N, 3)
    pixels_xyz = np.stack([X, Y, Z], axis=-1).reshape(-1, 3)
    # Aplicar la matriz DEF
    cohen = pixels_xyz @ MATRIZ_toDEF.T  # Resultado (N, 3)
    return cohen.reshape(alto, ancho, 3)

def calc_bch_to_xyz(B, C, H):
    alto, ancho = B.shape
    # Stack y reshape en una matriz (N, 3)
    pixels_bch = np.stack([B, C, H], axis=-1).reshape(-1, 3)
    # Aplicar la matriz XYZ
    xyz = pixels_bch @ MATRIZ_toXYZ.T  # Resultado (N, 3)
    return xyz.reshape(alto, ancho, 3)

def obtener_brillo_valores_def(X, Y, Z):
    cohen = calc_metricas_cohen(X, Y, Z)
    D, E, F = cohen[:, :, 0], cohen[:, :, 1], cohen[:, :, 2]
    B = np.sqrt(D**2 + E**2 + F**2)
    return B.mean(), D, E, F

### 🧮 Obtener brillo y canal DEF de la imagen

La función `obtener_brillo_imagen` aplica las funciones creadas en la celda anterior y devuelve el valor del brillo.

La función `obtener_canal_def` aplica las funciones creadas en la celda anterior y devuelve el valor del canal DEF.

**¿Cómo funciona?**  
- Separa la imagen en RGB
- Le pasa cada componente a la función obtener brillo
- Devuelve el brillo de la imagen y el canal DEF en las respectivas funciones

In [ ]:
def obtener_def_imagen(imagen_rgb):
    imagen_float = imagen_rgb.astype(np.float32) / 255.0

    X, Y, Z = cv2.split(cv2.cvtColor(imagen_float, cv2.COLOR_RGB2XYZ))
    
    _, D, E, F = obtener_brillo_valores_def(X, Y, Z)
    return D, E, F

### 🧮 Cargar imagen RGB   

La función `cargar_imagen_rgb` es una función auxiliar que carga una imagen en rgb .  


In [ ]:
def cargar_imagen_rgb(ruta):
    img = cv2.imread(ruta)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen: {ruta}")
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

### 🧮 Modificar contraste de una imagen   

La función `modificar_contraste` permite ajustar el contraste de una imagen.  

**¿Cómo funciona?**  
- Se determina un coeficiente ventana al llamar a la función que establece la cantidad de píxeles que se utilizarán para calcular la media del brillo.
- Se determina un coeficiente k al llamar a la función que establece la intensidad del contraste.
- Se calcula el brillo de la imagen utilizando la matriz BCH.
- Se obtiene el brillo promedio de la imagen.
- Se aplica la formula para modificar el contraste.
- Se modifican los canales de la imagen con los nuevos valores.
- Devuelve la imagen con el nuevo contraste.

In [ ]:
def calcular_b_promedio(imagen_rgb, ventana=15):
    """
    Calcula el brillo promedio en un vecindario para cada píxel
    
    Args:
        imagen_rgb: Imagen en formato RGB
        ventana: Tamaño de la ventana para el promediado (ventana x ventana)
    
    Returns:
        Matriz con el brillo promedio para cada píxel
    """
    # Obtener componentes D, E, F
    D, E, F = obtener_def_imagen(imagen_rgb)
    
    # Calcular brillo como la norma euclidiana de D, E, F
    brillo = np.sqrt(D**2 + E**2 + F**2)
    
    # Aplicar filtro de promedio con la ventana especificada
    kernel = np.ones((ventana, ventana), np.float32) / (ventana * ventana)
    b_promedio = cv2.filter2D(brillo, -1, kernel)
    
    return b_promedio

### 🧮 Modificar contraste

In [ ]:
def modificar_contraste(imagen_rgb, k, ventana=15):
    """
    Implementa la mejora de contraste preservando las coordenadas cromáticas
    según la fórmula (9) del paper
    """
    # Obtener componentes D, E, F
    D, E, F = obtener_def_imagen(imagen_rgb)
    
    # Calcular brillo como la norma euclidiana de D, E, F
    epsilon = 1e-10
    brillo = np.sqrt(D**2 + E**2 + F**2)
    
    # Calcular C y H según las fórmulas del modelo BCH
    # C = arccos(D/B)
    C = np.arccos(np.clip(D / (brillo + epsilon), -1.0, 1.0))
    
    # H = arctan2(F, E) - Usando arctan2 para obtener el ángulo completo en el rango [-π, π]
    sin_C = np.sin(C)
    # Evitar división por cero
    mask = sin_C > epsilon
    H = np.zeros_like(C)
    
    # E = B*sin(C)*cos(H),  F = B*sin(C)*sin(H)
    # tan(H) = F/E  →  H = arctan2(F, E)
    # Usar arctan2 para determinar el ángulo correctamente en todos los cuadrantes
    H[mask] = np.arctan2(F[mask], E[mask])
    
    # Calcular luminosidad promedio en el vecindario
    b_promedio = calcular_b_promedio(imagen_rgb, ventana)
    
    # Aplicar la fórmula de mejora de contraste
    ratio = brillo / (b_promedio + epsilon)
    nuevo_B = b_promedio * np.power(ratio, k)
    
    # Limitar valores al rango válido
    nuevo_B = np.clip(nuevo_B, 0, 255)
    
    # Reconstruir D, E, F usando las fórmulas del modelo BCH
    # D = B * cos(C)
    nuevo_D = nuevo_B * np.cos(C)
    # E = B * sin(C) * cos(H)
    nuevo_E = nuevo_B * np.sin(C) * np.cos(H)
    # F = B * sin(C) * sin(H)
    nuevo_F = nuevo_B * np.sin(C) * np.sin(H)
    
    # Convertir de DEF a XYZ
    nuevoXYZ = calc_bch_to_xyz(nuevo_D, nuevo_E, nuevo_F)
    
    # Asegurar que nuevoXYZ esté en el formato correcto
    nuevoXYZ = nuevoXYZ.astype(np.float32)
    
    # Convertir de vuelta a RGB
    resultado = cv2.cvtColor(nuevoXYZ, cv2.COLOR_XYZ2RGB) * 255
    
    # Retornar como uint8
    return np.clip(resultado, 0, 255).astype(np.uint8)


### 🗂️ Procesamiento masivo de imágenes modificación del contraste

Esta función `procesar_imagenes_contraste` aplica la métrica de Cohen para modificar el contraste a todas las imágenes `.png` dentro de una estructura de carpetas.

**Qué hace:**
- Recorre subcarpetas dentro de una carpeta principal.
- Carga la imagen en RGB.
- Aplica la función `modificar_contraste` para editar la imagen.
- Guarda cada una de las imagenes modificadas.

In [13]:
def procesar_imagenes_contraste(
    carpeta_entrada="imagenes",
    carpeta_salida="imagenes_procesadas",
    valores_k=[0.5, 1.0, 2.0, 3.0, 4.0, 5.0]
):

    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)
        
        # Verificamos si es una carpeta
        if not os.path.isdir(ruta_subcarpeta):
            continue
        
        # Crear subcarpeta de salida correspondiente
        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta)
        # Añadimos la subcarpeta específica para 'contraste'
        carpeta_salida_sub = os.path.join(carpeta_salida_sub, "contraste")
        os.makedirs(carpeta_salida_sub, exist_ok=True)
                    
        # Recorremos los archivos dentro de la subcarpeta
        for filename in sorted(os.listdir(ruta_subcarpeta)):
            if filename.lower().endswith("low_contrast.png"):
                ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)

                # Cargar imagen
                img_rgb = cargar_imagen_rgb(ruta_imagen_entrada)
                if img_rgb is None:
                    continue
                
                nombre, _ = os.path.splitext(filename)

                # Filtro
                for k in valores_k:
                    contraste_mejorado = modificar_contraste(img_rgb, k)
                    
                    nombre_salida = f"{nombre}_k{k:.1f}.jpg"
                    ruta_imagen_salida = os.path.join(carpeta_salida_sub, nombre_salida)

                    # Convertir de RGB a BGR para guardar con OpenCV
                    contraste_mejorado_bgr = cv2.cvtColor(contraste_mejorado, cv2.COLOR_RGB2BGR)
                    cv2.imwrite(ruta_imagen_salida, contraste_mejorado_bgr)

    print("Procesamiento completado con mejora de contraste.")

In [14]:
# Ejecuta el procesamiento
procesar_imagenes_contraste(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas",
    valores_k=[0.5, 1.0, 2.0, 3.0, 4.0, 5.0]
)

NameError: name 'os' is not defined

### 🧮 Calcular varianza histograma  
Esta función `calcular_varianza_histograma` calcula la varianza del histograma de una imagen en escala de grises.

**Qué hace:**
- Recibe una imagen en escala de gises y calcula su hisotgrama.
- Calcula la varianza
- Devuelve tanto la varianza como el histograma

In [17]:
#Medir contraste
def calcular_varianza_histograma(img_gray):
    """
    Calcula la varianza en el histograma para saber el contraste de la imagen 
    Retorna:
        - Varianza (dispersión de intensidades).
    """
       
    # Calcular histograma
    hist = cv2.calcHist([img_gray], [0], None, [256], [0, 256]).flatten()
    
    total_pixeles = img_gray.size
    valores = np.arange(256) # Valores posibles de intensidad (0-255)
    media = np.sum(valores * hist) / total_pixeles
    varianza = np.sum(((valores - media) ** 2) * hist) / total_pixeles

    return hist, varianza

### 🗂️ Procesamiento masivo de imágenes información contraste

Esta función `procesar_imagenes_info_contraste` calcula la varianza del histograma a todas las imágenes `.png` dentro de una estructura de carpetas.

**Qué hace:**
- Recorre subcarpetas dentro de una carpeta principal.
- Carga la imagen en escala de grises.
- Aplica la función `calcular_varianza_histograma` para calcular la varianza de cada imagen.
- Guarda el valor de cada varianza en un archivo de texto plano

In [15]:
def procesar_imagenes_info_contraste(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas"
):
    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)

        if not os.path.isdir(ruta_subcarpeta):
            continue

        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta, "contraste")
        os.makedirs(carpeta_salida_sub, exist_ok=True)

        ruta_txt = os.path.join(carpeta_salida_sub, "info_varianzas_contraste.txt")
        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write("Varianzas de los histogramas en escala de grises de las imágenes\n")
            archivo_txt.write(f"Carpeta de imágenes: {ruta_subcarpeta}\n\n")
            archivo_txt.write("La varianza indica la dispersión de los niveles de gris.\n")
            archivo_txt.write("Valores mayores de la varianza corresponden a mayor contraste.\n\n")

            for filename in sorted(os.listdir(ruta_subcarpeta)):
                if not filename.lower().endswith(("sharp.png", "contrast.png")):
                    continue

                ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)
                img_bgr = cv2.imread(ruta_imagen_entrada)
                if img_bgr is None:
                    continue

                img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

                # Calcular la varianza del histograma
                hist, varianza = calcular_varianza_histograma(img_gray)

                nombre, _ = os.path.splitext(filename)
                nombre_salida = f"{nombre}_histograma_grises.png"
                ruta_imagen_salida = os.path.join(carpeta_salida_sub, nombre_salida)

                # Graficar y guardar el histograma
                plt.figure()
                plt.plot(hist, color='black')
                plt.title(f"Histograma de {filename}")
                plt.xlabel("Intensidad de píxeles")
                plt.ylabel("Frecuencia")
                plt.savefig(ruta_imagen_salida)
                plt.close()
                
                archivo_txt.write(f"{filename} -> varianza: {varianza:.5f}\n")

    print("Procesamiento completado con cálculo de contraste.")

In [16]:
procesar_imagenes_info_contraste(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas"
)

NameError: name 'os' is not defined